# Reaction Classification Project - Dataset Discovery

**Goal:** Find and explore ORD datasets for Suzuki-Miyaura coupling and C-N coupling reactions to build a reaction classifier.

---

In [ ]:
# Core imports
from ord_schema import message_helpers
from ord_schema.proto import dataset_pb2, reaction_pb2
from glob import glob
import gzip
import os
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

## 1. Scan ORD Repository for Datasets

First, let's find all available datasets and check their names/descriptions for our target reaction types.

In [ ]:
# Set path to your ord-data repository
# UPDATE THIS PATH if your repo is in a different location
ORD_DATA_PATH = "ord-data/data"

# Find all dataset files
dataset_files = glob(os.path.join(ORD_DATA_PATH, "**/*.pb.gz"), recursive=True)
print(f"Found {len(dataset_files)} dataset files")

In [ ]:
# Keywords to search for in dataset names/descriptions
SUZUKI_KEYWORDS = ["suzuki", "miyaura", "suzuki-miyaura", "boronic", "boronate"]
CN_KEYWORDS = ["c-n coupling", "c-n bond", "buchwald", "hartwig", "amination", "ullmann"]

def check_keywords(text, keywords):
    """Check if any keyword is in the text (case-insensitive)."""
    if text is None:
        return False
    text_lower = text.lower()
    return any(kw in text_lower for kw in keywords)

def get_dataset_info(filepath):
    """Load dataset and extract basic info."""
    try:
        dataset = message_helpers.load_message(filepath, dataset_pb2.Dataset)
        return {
            "filepath": filepath,
            "dataset_id": dataset.dataset_id,
            "name": dataset.name,
            "description": dataset.description,
            "num_reactions": len(dataset.reactions)
        }
    except Exception as e:
        return None

In [ ]:
# Scan all datasets and categorize
suzuki_datasets = []
cn_datasets = []
other_datasets = []

print("Scanning datasets (this may take a few minutes)...")

for filepath in tqdm(dataset_files):
    info = get_dataset_info(filepath)
    if info is None:
        continue
    
    # Check name and description for keywords
    text_to_check = f"{info['name']} {info['description']}"
    
    if check_keywords(text_to_check, SUZUKI_KEYWORDS):
        suzuki_datasets.append(info)
    elif check_keywords(text_to_check, CN_KEYWORDS):
        cn_datasets.append(info)
    else:
        other_datasets.append(info)

print(f"\nFound:")
print(f"  - Suzuki-Miyaura datasets: {len(suzuki_datasets)}")
print(f"  - C-N coupling datasets: {len(cn_datasets)}")
print(f"  - Other datasets: {len(other_datasets)}")

## 2. Explore Found Datasets

In [ ]:
# Display Suzuki-Miyaura datasets
print("=" * 80)
print("SUZUKI-MIYAURA DATASETS")
print("=" * 80)

suzuki_df = pd.DataFrame(suzuki_datasets)
if len(suzuki_df) > 0:
    suzuki_df = suzuki_df.sort_values("num_reactions", ascending=False)
    for i, row in suzuki_df.iterrows():
        print(f"\n📁 {row['name'][:80]}..." if len(str(row['name'])) > 80 else f"\n📁 {row['name']}")
        print(f"   Reactions: {row['num_reactions']}")
        print(f"   ID: {row['dataset_id']}")
        desc = str(row['description'])[:150] + "..." if len(str(row['description'])) > 150 else row['description']
        print(f"   Description: {desc}")
else:
    print("No Suzuki-Miyaura datasets found with current keywords.")

In [ ]:
# Display C-N coupling datasets
print("=" * 80)
print("C-N COUPLING DATASETS")
print("=" * 80)

cn_df = pd.DataFrame(cn_datasets)
if len(cn_df) > 0:
    cn_df = cn_df.sort_values("num_reactions", ascending=False)
    for i, row in cn_df.iterrows():
        print(f"\n📁 {row['name'][:80]}..." if len(str(row['name'])) > 80 else f"\n📁 {row['name']}")
        print(f"   Reactions: {row['num_reactions']}")
        print(f"   ID: {row['dataset_id']}")
        desc = str(row['description'])[:150] + "..." if len(str(row['description'])) > 150 else row['description']
        print(f"   Description: {desc}")
else:
    print("No C-N coupling datasets found with current keywords.")

In [ ]:
# Summary statistics
print("\n" + "=" * 80)
print("SUMMARY")
print("=" * 80)

total_suzuki = sum(d['num_reactions'] for d in suzuki_datasets)
total_cn = sum(d['num_reactions'] for d in cn_datasets)

print(f"\nSuzuki-Miyaura:")
print(f"  - {len(suzuki_datasets)} datasets")
print(f"  - {total_suzuki} total reactions")

print(f"\nC-N Coupling:")
print(f"  - {len(cn_datasets)} datasets")
print(f"  - {total_cn} total reactions")

print(f"\nCombined: {total_suzuki + total_cn} reactions for classification")

## 3. Select Datasets for Training

Based on the above exploration, select which datasets to use. Ideally pick datasets that:
- Have enough reactions (>100 each)
- Have yield data available
- Are reasonably balanced between classes

In [ ]:
# After reviewing the output above, manually select the datasets you want to use
# Fill in the filepaths from the discovered datasets

SELECTED_SUZUKI = [
    # Add filepaths here, e.g.:
    # "ord-data/data/xx/ord_dataset-xxxxx.pb.gz",
]

SELECTED_CN = [
    # Add filepaths here, e.g.:
    # "ord-data/data/xx/ord_dataset-xxxxx.pb.gz",
]

print(f"Selected {len(SELECTED_SUZUKI)} Suzuki datasets")
print(f"Selected {len(SELECTED_CN)} C-N coupling datasets")

## 4. Deep Dive: Inspect a Dataset

Before committing, let's look at the structure of a specific dataset to understand what features are available.

In [ ]:
def inspect_dataset(filepath, n_samples=3):
    """Inspect the structure and content of a dataset."""
    dataset = message_helpers.load_message(filepath, dataset_pb2.Dataset)
    
    print(f"Dataset: {dataset.name}")
    print(f"Total reactions: {len(dataset.reactions)}")
    print("\n" + "-" * 40)
    
    # Check what fields are populated across reactions
    has_yield = 0
    has_temperature = 0
    has_time = 0
    input_roles = defaultdict(int)
    
    for rxn in dataset.reactions:
        # Check for yield
        for outcome in rxn.outcomes:
            for product in outcome.products:
                for m in product.measurements:
                    if m.type == reaction_pb2.ProductMeasurement.YIELD:
                        has_yield += 1
                        break
        
        # Check conditions
        if rxn.conditions.HasField("temperature"):
            has_temperature += 1
        
        # Check for reaction time in outcomes
        for outcome in rxn.outcomes:
            if outcome.HasField("reaction_time"):
                has_time += 1
                break
        
        # Count input roles
        for key, inp in rxn.inputs.items():
            for comp in inp.components:
                role = reaction_pb2.ReactionRole.ReactionRoleType.Name(comp.reaction_role)
                input_roles[role] += 1
    
    n = len(dataset.reactions)
    print(f"\nData availability:")
    print(f"  - Yield data: {has_yield}/{n} ({100*has_yield/n:.1f}%)")
    print(f"  - Temperature: {has_temperature}/{n} ({100*has_temperature/n:.1f}%)")
    print(f"  - Reaction time: {has_time}/{n} ({100*has_time/n:.1f}%)")
    
    print(f"\nInput roles found:")
    for role, count in sorted(input_roles.items(), key=lambda x: -x[1]):
        print(f"  - {role}: {count}")
    
    # Show sample reactions
    print(f"\n" + "=" * 40)
    print(f"Sample reactions:")
    print("=" * 40)
    
    for i, rxn in enumerate(dataset.reactions[:n_samples]):
        print(f"\n--- Reaction {i+1} ---")
        for key, inp in rxn.inputs.items():
            print(f"  Input '{key}':")
            for comp in inp.components:
                role = reaction_pb2.ReactionRole.ReactionRoleType.Name(comp.reaction_role)
                # Get identifier
                ident_str = "[no identifier]"
                for ident in comp.identifiers:
                    ident_type = reaction_pb2.CompoundIdentifier.CompoundIdentifierType.Name(ident.type)
                    if ident_type in ["SMILES", "NAME"]:
                        ident_str = f"{ident_type}: {ident.value[:50]}{'...' if len(ident.value) > 50 else ''}"
                        break
                print(f"    - {role}: {ident_str}")
    
    return dataset

In [ ]:
# Inspect one of the found Suzuki datasets (if any)
if len(suzuki_datasets) > 0:
    # Pick the largest one
    largest_suzuki = max(suzuki_datasets, key=lambda x: x['num_reactions'])
    print("Inspecting largest Suzuki-Miyaura dataset:\n")
    _ = inspect_dataset(largest_suzuki['filepath'])

In [ ]:
# Inspect one of the found C-N coupling datasets (if any)
if len(cn_datasets) > 0:
    # Pick the largest one
    largest_cn = max(cn_datasets, key=lambda x: x['num_reactions'])
    print("Inspecting largest C-N coupling dataset:\n")
    _ = inspect_dataset(largest_cn['filepath'])

## 5. Next Steps

After running this notebook, you will know:
1. How many datasets exist for each reaction type
2. How many reactions are available
3. What features are consistently available (yield, temperature, roles, etc.)

The next notebook will:
1. Load the selected datasets
2. Extract features (SMILES → fingerprints or one-hot encoding)
3. Build and train the classification model

---

**Note:** If the keyword search doesn't find enough datasets, we may need to:
- Expand the keywords
- Look at the reaction SMILES patterns directly
- Check the `reaction_type` field in the reaction messages